# 第13回 データの可視化

___
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tsuboshun/begin-python/blob/gh-pages/_sources/workbook/lecture13.ipynb)

___

## この授業で学ぶこと

今回から第15回にかけてデータサイエンスの入門講義を行う。
今回はデータの可視化をテーマに、matplotlibというライブラリの使い方を一通り解説する。またテーブルデータを扱うためのライブラリであるpandasについても解説する。

今回の授業で紹介するコードは何も見ないで書けるようになる必要はなく、これらを雛形にして使えるようになればよい。
matplotlibを使うと綺麗なグラフを作成できるようになるので、ぜひ他の授業のレポートなどでも活用してみてほしい。
また今回から課題を解くのにChatGPTを活用してもよいことにする。

## matplotlib入門

Pythonでグラフを描画するのに**matplotlib**というライブラリがよく用いられる。この節ではmatplotlibの使い方の基本を説明する。

matplotlibを使うには、その中のpyplotというモジュールをimportする。
またグラフのラベルに日本語を使えるようにするためにjapanize_matplotlibというモジュールをimportする。japanize_matplotlibはpipによるインストールが必要である[^f1]。

[^f1]: matplotlibも標準ライブラリではないため本来はpipによるインストールが必要であるが、Google Colabの環境では初めからインストールされている。

In [ ]:
pip install japanize_matplotlib

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib
plt.rcParams.update({'font.size': 14})  # 文字サイズを14pxとする

まずは描画用のサンプルデータを用意しよう。matplotlibではリストまたは[第6回で紹介](label:numpy)したNumPyの配列（`ndarray`）を扱うことができる。ここでは配列のデータを用意する。

In [ ]:
import numpy as np

# 0から10までの0.1刻みの配列
sample_x = np.arange(0, 10, 0.1)

# y = 2x + eps　（epsはノイズ）
eps = np.random.normal(loc=0, scale=1, size=len(sample_x))
sample_y = sample_x * 2 + eps

`np.arange()` 関数は、`np.arange(start, stop, step)` と呼び出すと `start` から `stop` までの `step` 刻みの配列を生成する。

In [ ]:
sample_x

In [ ]:
len(sample_x)

`np.random.normal()` 関数は上のように呼び出すと、平均が0で標準偏差が1の正規分布に従う乱数の配列（長さ `len(sample_x)`）を生成する。代入先の `eps` はデータに加えるノイズとして使用する。

In [ ]:
eps

配列の特徴の1つは、要素ごとの演算を一括で行えることである。`sample_x * 2 + eps` により、インデックスごとに `sample_x` の要素を2倍し、`eps` の要素を足している。

ここで生成した `sample_x` と `sample_y` は $\varepsilon$ をノイズとして $y = 2x + \varepsilon$ の関係を満たす2次元データ $(x, y)$ を100個生成したものと見ることができる。
1つ1つのデータは(`sample_x[0]`, `sample_y[0]`), (`sample_x[1]`, `sample_y[1]`), ..., (`sample_x[99]`, `sample_y[99]`)である。

### 散布図

サンプルデータを**散布図**として可視化してみよう。横軸に $x$、縦軸に $y$ の値をとって各データを点として打って描画する。このことをデータを散布図として**プロット**するという。

In [ ]:
# プロットの入れ物の用意
fig, ax = plt.subplots(1, 1, figsize=(5, 4))

# 散布図のプロット
ax.scatter(sample_x, sample_y)

# 表示
plt.show()

1行目では `plt.subplots()` 関数によりFigureオブジェクトとAxesオブジェクトを生成して、変数 `fig` と `ax` に代入している[^f2][^f3]。

matplotlibによる描画はFigureオブジェクトとAxesオブジェクト、Axisオブジェクトの3つの要素で構成される。Axisオブジェクトは1つの軸を管理し、Axesオブジェクトは1つのグラフを管理し、Figureオブジェクトは描画全体を管理する。それぞれの関係を次の図に示す。

[^f2]: 一見すると戻り値が2つあるように見えるが、`plt.subplots()` はFigureオブジェクトとAxesオブジェクトのタプルを返しており（つまり戻り値は1つ）、タプルの各要素を `fig` と `ax` に割り当てている。戻り値のタプルを `,` 区切りの変数で受け取ることを**アンパック**という。
[^f3]: matplotlibの描画方法にはexplicit（明示的）な方法とimplicit（非明示的）な方法の2種類があるが、このテキストではexplicitな方法を説明する。implicitな方法というのは、`fig` や `ax` を用意することなく、いきなり `plt.plot()` などとプロットする方法のことである。implicitな方法は手軽にプロットできるというメリットがあるが、グラフの細かい調整ができないというデメリットもある。そのため、はじめからexplicitな方法を覚えるのがお勧めである。

```{figure} ./pic/fig_map.png
---
width: 400px
name: fig_map
---
グラフの構成要素
```

`plt.subplots()` 関数は `plt.subplots(n, m, figsize=(w, h))` と呼び出すと、横幅 `w`、高さ `h` （単位はインチ）のFigureオブジェクトと、その中に `n` 行 `m` 列の`n * m` 個のAxesオブジェクトを作成する。グラフを1枚だけ描画したい場合は、`n = m = 1` と指定する。または引数で何も指定しなければ、デフォルト引数により `n = m = 1` が設定される。このときAxesオブジェクトが1つだけ作成され、戻り値の `ax` はAxesオブジェクトそのものになる。複数のグラフを同時に描画したい場合は、`n` または `m` または両方に2以上の整数を指定する。このときAxesオブジェクトが複数作成され、戻り値はAxesオブジェクトの配列となる。

散布図のプロットは `ax.scatter()` メソッドにより行う。第一引数に横軸の値の配列、第二引数に縦軸の値の配列を指定する。
最後に `plt.show()` 関数を呼び出すことで、グラフが表示される。

### グラフの調整

次にグラフの見た目を細かく調整してみよう。

まずは1枚のグラフに複数の散布図をプロットする。
先ほどのプロットに $y = 3x + \varepsilon$ の関係を満たすデータの散布図を追加する。

In [ ]:
# プロットの入れ物の用意
fig, ax = plt.subplots(figsize=(5, 4))

# データの用意
sample_y2 = sample_x * 3 + np.random.normal(loc=0, scale=1, size=len(sample_x))

# 散布図のプロット
ax.scatter(sample_x, sample_y)
ax.scatter(sample_x, sample_y2)

# 表示
plt.show()

このように同一のAxesオブジェクトの `scatter()` メソッドを複数回呼び出すことで、1つのグラフに複数の散布図をプロットすることができる。

プロットの色は自動的に設定されるが、自分で指定することもできる。
凡例や軸ラベル、タイトル、マーカーの種類の設定方法とともに紹介する。

In [ ]:
# プロットの入れ物の用意
fig, ax = plt.subplots(figsize=(5, 4))

# 散布図のプロット
ax.scatter(sample_x, sample_y, color='royalblue', marker='v', label='y=2x+e')
ax.scatter(sample_x, sample_y2, color='forestgreen', marker='x', label='y=3x+e')

# 軸ラベル・タイトルの設定
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('xとyの関係')

# 表示
plt.legend() # 凡例の表示
plt.show()

`ax.scatter()` メソッドのキーワード引数を使って、`color` を設定すると色を変更できる（`color` で指定できる色の名前の一覧は[こちら](https://matplotlib.org/stable/gallery/color/named_colors.html)）。`marker` を設定するとマーカーの種類を変更できる（`marker` で指定できる値の一覧は[こちら](https://matplotlib.org/stable/api/markers_api.html)）。`label` を設定すると凡例の表示を設定できる。凡例を表示するには、最後に `plt.legend()` を呼び出す。

軸ラベルは `ax.set_xlabel()`、`ax.set_ylabel()` により設定する。同様にタイトルは `ax.set_title()` により設定する。

### 折れ線グラフ

散布図と並んでよく使われるのが**折れ線グラフ**である。折れ線グラフでは、基本的にデータの順番は $x$ の値について昇順または降順であることを仮定する。
このとき横軸に $x$、縦軸に $y$ の値をとって各データを点として打つところは散布図と同じであるが、折れ線グラフではこれらの点を線で結ぶ。これにより $x$ の値が増えたときの、$y$ の値の連続的な変化を表現できる。

折れ線グラフのプロット例を以下に示す。
ノイズのない $y = 2x$ の関係を表す2次元データ $(x, y)$ を `sample_x`、`sample_y3` に用意し、その折れ線グラフを最初の散布図に重ねてプロットしている。

In [ ]:
# プロットの入れ物の用意
fig, ax = plt.subplots(figsize=(5, 4))

sample_y3 = sample_x * 2

# 散布図、折れ線グラフのプロット
ax.scatter(sample_x, sample_y)
ax.plot(sample_x, sample_y3, color="orange")

# 表示
plt.show()

## pandas入門

Pythonで表形式のデータ（テーブルデータ）を扱うのに**pandas**というライブラリがよく用いられる。この節ではpandasについて、今後の授業で使いうる機能に限定して説明する。

pandasを使うには、pandasモジュールをimportする。
ここでは説明のため、seabornというライブラリにサンプルとして用意されている `'diamonds'` というテーブルデータを使用する（データの説明は[こちら](https://ggplot2.tidyverse.org/reference/diamonds.html)）。`load_dataset()` 関数によりサンプルのテーブルデータを取得できる。

In [ ]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('diamonds')

### データフレーム

pandasにおいてテーブルデータのことを**データフレーム**（**DataFrame**）という。
データフレームの持つ `head()` メソッドにより、先頭から5件のデータを表示することができる。
`head(10)` のように引数に整数を渡すことで、その件数だけデータを表示することもできる。

In [ ]:
df.head()

データフレームは、2次元配列の行方向と列方向にラベルがついたものと理解することができる。行方向のラベルのことを**index**、列方向のラベルのことを**columns**という。indexとcolumnsの値を指定して、要素にアクセスするには `loc` を使って次のように書く。

In [ ]:
df.loc[0, "carat"]

データフレームは2次元配列と相互に変換することができる。データフレームから2次元配列を取得するには、`values` 属性にアクセスすればよい。

In [ ]:
array = df.values
array

2次元配列からデータフレームを作成するには次のように書く。

In [ ]:
df2 = pd.DataFrame(array)
df2.head()

columnsのデフォルト値は0始まりの整数である。データフレームの作成時にcolumnsを設定するには、次のように書く。

In [ ]:
df2 = pd.DataFrame(array, columns=['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z'])
df2.head()

### シリーズ

データフレームは2次元配列に行ラベル・列ラベルをつけたものであった。
pandasには、1次元配列に行ラベルをつけたデータ型も用意されており、**シリーズ**（**Series**）という。

データフレームに対してcolumnsの値を `[]` で指定すると、その列のデータをシリーズとして取得できる。

In [ ]:
s = df["carat"]
s.head()

indexの値を指定して、要素にアクセスするには `loc` を使って次のように書く。

In [ ]:
s.loc[0]

シリーズは1次元配列と相互に変換することができる。シリーズから1次元配列を取得するには、`values` 属性にアクセスすればよい。

In [ ]:
array = s.values
array

1次元配列からシリーズを作成するには次のように書く。

In [ ]:
s2 = pd.Series(array)
s2.head()

### テーブル操作

データフレームを使うと、テーブルデータでよく行われる操作を簡単に実現することができる。データフレーム上でテーブルデータを加工し、加工したテーブルデータの値を配列として取得して、matplotlibなど他のライブラリに渡すというのがよくある使い方である。以下にいくつかの操作を紹介する。

#### ソート

`sort_values()` メソッドにより、引数に渡した列を基準にソートすることができる。デフォルトでは昇順にソートする。ソートした結果のデータフレームが戻り値として返されるので、以下のコードではそれを変数 `df` に再代入している。

In [ ]:
df = df.sort_values("price")
df.head()

降順にソートしたい場合は、引数に `ascending=False` を指定する。

In [ ]:
df = df.sort_values("price", ascending=False)
df.head()

#### フィルタリング

テーブルデータの中から特定の条件を満たす行を抽出することを**フィルタリング**という。

フィルタリングを行うには、どのindexの行を抽出するかをブール型で表したシリーズを `loc` で指定する。
例えば、`cut` の種類が `Premium` の行を抽出したいとしよう。
このとき `cut` の種類が `Premium` かどうかを表すシリーズを次のように作成できる。

In [ ]:
df['cut'] == 'Premium'

このシリーズを `loc`で指定することで、データフレームから `cut` の種類が `Premium` の行のみ抽出することができる。

In [ ]:
df_premium = df.loc[df['cut'] == 'Premium']
df_premium.head()

#### 列の演算

シリーズは、配列と同様に各種演算を行うことができる。またデータフレームの列の追加や更新は代入文により行うことができる。
例えば、`x`、`y`、`z` の総和を求めて、その結果を `sum` という列として保持するには、以下のように書く。

In [ ]:
df['sum'] = df['x'] + df['y'] + df['z']
df.head()

例えば、`price` の対数を計算して `price_log` という列として保持するには、以下のように書く。
このようにデータが数値の列に対して、NumPyの関数を適用することも可能である。

In [ ]:
df['price_log'] = np.log(df['price'])
df.head()

## ヒストグラム

データの可視化の最後の例として**ヒストグラム**を紹介する。
ヒストグラムはデータの値をいくつかの階級に分割し、階級ごとのデータ数を集計して可視化する。
ヒストグラムによりデータの**分布**を確認することができる。

ヒストグラムのプロット例を以下に示す。先ほどの `diamonds` データにおける `"price"` の値を1次元配列として取得し、そのヒストグラムをプロットしている。

In [ ]:
# プロットの入れ物の用意
fig, ax = plt.subplots(figsize=(5, 4))

# データの用意
x = df["price"].values

# ヒストグラムのプロット
ax.hist(x, bins=10)

# 軸ラベルの設定
ax.set_xlabel("価格")
ax.set_ylabel("カウント")

# 表示
plt.show()

Axesの `hist()` メソッドに1次元配列を渡すことで、ヒストグラムを作成できる。キーワード引数で `bins` を指定すると、横軸の分割数を指定できる（**ビン数**という）。ビン数を大きくすると、より細かい粒度で分布の形を確認できる。
上のヒストグラムからは、例えば価格の低いダイヤモンドにデータ数が偏っていることが確認できる。

上の例では価格という1つの軸をもとにデータを分割したが、2つの軸をもとにデータを分割してデータ数を確認することもできる。
そのようなヒストグラムを**2次元ヒストグラム**という。

2次元ヒストグラムのプロット例を以下に示す。
横軸に `"carat"` の対数の値を、縦軸に `"price"` の対数の値をとり、それ上のデータ数を**カラープロット**により表現している。

In [ ]:
# プロットの入れ物の用意
fig, ax = plt.subplots(figsize=(5, 4))

# データの用意
x = np.log(df["carat"].values)
y = np.log(df["price"].values)

# 2次元ヒストグラムのプロット
hb = ax.hexbin(x, y, gridsize=10)

# カラーバーのプロット
cb = fig.colorbar(hb, ax=ax)
cb.set_label('カウント')

# 表示
plt.show()

ヒストグラムの階級を作るにあたって平面を六角形で分割している。このような区分を**Hexbin**という。
色とデータ数の対応関係を表すのが、右横の**カラーバー**である。
カラーバーは　`#カラーバーのプロット` 以下の2行によりプロットできる。

## 演習